# R Workbook 2: Using Joins, aggregations, and visualizations

In the previous workbook, we looked at how to read in a dataset, create a data frame, and then explore it. In this workbook, we'll explore how to join 2 data frames. In addition, we'll look at using basic visualizations.

## <span style = "color:green">Motivating Question</span>

So far, we've gone over basic R operations such as subsetting data, describing data, and reading in CSV files. However, part of what makes R so useful is its powerful tools and functionality. There are many insights we might find from properly analyzing data using joins, aggregrations, and visualizing the reusults, so we want to be able to bring that all together. In this notebook, we'll explore how to do that as we try to answer the following questions:

**What is the difference in the number of jobs by county? What are the job proportions for each county?**

To answer these, we'll introduce you to joins along with aggregration functions. Finally, we'll show some basic visualizations with the data that we get to bring it all together.

## Starting Out
As before, we'll start by loading libraries. We have a few more commands we'll be running in the beginning since we're also doing visualizations.

In [ ]:
library(tidyverse)
library(dplyr)
options(scipen = 999)

# Visualization settings to make them look nicer in Jupyter
theme_set(theme_gray(base_size = 20))
options(repr.plot.width = 12, repr.plot.height = 9)

Read in hints and solutions to the questions. Throughout this notebook you will see cells that contain .hint() and .solution(). You need to un-comment the line to run these cells. 

In [ ]:
# Import the file with hints and solutions
source("r2_hints_and_solutions.txt")

Let's read in our datasets and take a quick glance using the `head` function. 

In [ ]:
ca_wac_2015 <- read.csv("ca_wac_S000_JT00_2015.csv")
ca_xwalk <- read.csv("ca_xwalk.csv")


To quickly explore the read in tables, try running `head()` or `glimpse()`.

## <span style="color:green">Joins</span>

One of the nice things about relational databases is organization using multiple tables that are linked together in some way. For example, suppose we have one table with 6 rows called **Table A**:

| blockid | C000|
|---|---|
|1|5|
|2|10|
|3|2|
|4|6|
|5|22|
|6|9|

And another table with 5 rows called **Table B**:

| blockid | CA01|
|---|---|
|2|2|
|5|4|
|6|1|
|7|2|
|8|0|

Let's say we want to combine Table A and Table B so that we have one table that contains information about `blockid`, `C000`, and `CA01`. We want to do this by matching the two tables by what they have in common, `blockid`. That is, we want a table that looks like this (let's call this **Table C**):

| blockid | C000 | CA01 |
|---|---|---|
|2|10|2|
|5|22|4|
|6|9|1|

Table C has each `blockid` that was in both Table A and Table B. It also contains the appropriate values for `C000` and `CA01` corresponding to each `blockid`. This kind of matching can be quite tricky to figure out manually, since there are different numbers of rows in each table, not all of the `blockid` values match for the two tables, and there are some `blockid` values that aren't in both. Fortunately for us, R is well-equipped to handle this task using the `inner_join` statement.

### R Code and how it works


Given the data we have available, when performing an `inner_join` on the WAC table to the geography crosswalk table, we would be able to determine counties or metropolitan/micropolitan areas for census block containing workplaces.

Now that we have established a plan for how we're joining two tables together, let's take a look at the R code that performs this join and break it down.

    df_inner_join <- inner_join(
    ca_xwalk, ca_wac_2015, 
    by = c("tabblk2010" = "w_geocode"))

Let's look at the first two lines.

    df_inner_join <- inner_join(
    ca_xwalk, ca_wac_2015, 

Here, we want to create a variable called `df_inner_join` which will store the results of the inner join, then we are calling the function `inner_join`. The second line takes the `ca_xwalk` table and joins the `ca_wac_2015` table to it.

We can't just mash two tables together though -- we need some way of making sure that the appropriate rows match. We do this with the third line:

    by = c("tabblk2010" = "w_geocode"))
    
This part specifies what we're joining on. That is, what is the ID variable that is in both tables that we want to match. Notice that they don't need to be named the same in both tables, though you do need to specify what they are in each table, even if they are the same.

If you run the full code below, you should see the full results of the joined table. You should be able to scroll through all of the variables and see that we've managed to merge the `ca_wac_2015` and `ca_xwalk` tables together according to their census block IDs.

> **Side note:** We're only going to be displaying a few of the columns which we can do using the `select` function. This is because we aren't able to display more than 50 columns here in this notebook format. Joining to get tables with greater than 50 columns is perfectly fine, but we'll only look at a few at a time to make it easier to follow in these exercises.

In [ ]:
df_inner_join <- inner_join(ca_xwalk, ca_wac_2015, by = c("tabblk2010" = "w_geocode")) %>%
    select (tabblk2010, C000, CA01, CA02, CA03, ctyname, cbsaname)
    
head(df_inner_join)

## Checkpoint 1: Get Census Data and Join with your Tables

Try joining the Residence Area Characteristics table with the Crosswalk table in a similar manner, as well as changing some of the columns to display. As you construct your query, make sure you answer the following question regarding the join:
- What are the top 5 counties that have the most total number of jobs per residence census block? Ansewr the following questions as you write the query:
    - What are they being joined on? That is, what is the "ID" variable you're matching on in each table?
    - What information does the resulting table give you? For example, after we join the WAC table to the geography crosswalk table, we are now able to determine counties or metropolitan/micropolitan areas for census block containing workplaces.
    > Hint: the Residence Area Characteristics table is titled `ca_rac_2015` and the Crosswalk table is titled `ca_xwalk`.



First, run the cell below to read in the `ca_rac_2015` table.

In [ ]:
ca_rac_2015 <- read.csv("ca_rac_S000_JT00_2015.csv")

In [ ]:
# checkpoint_1.hint()

In [ ]:
# checkpoint_1.solution()

## <span style="color:green">Different Types of Joins</span>

We've so far done only one type of join, an inner join. However, there are different types of joins.

### Left and Right Joins in R

Recall our **Table A**:

| blockid | C000|
|---|---|
|1|5|
|2|10|
|3|2|
|4|6|
|5|22|
|6|9|

And **Table B**:

| blockid | CA01|
|---|---|
|2|2|
|5|4|
|6|1|
|7|2|
|8|0|


Suppose we want to look at every single census block in one table, only filling in information from the second table if it exists. We'll illustrate this using Table A and Table B from before. Recall that our `inner_join` created Table C:

| blockid | C000 | CA01 |
|---|---|---|
|2|10|2|
|5|22|4|
|6|9|1|

Instead, we want to create the following table:

| blockid | C000 | CA01 |
|---|---|---|
|1|5|*null*|
|2|10|2|
|3|2|*null*|
|4|6|*null*|
|5|22|4|
|6|9|1|

Here, we've kept every single row in Table A, and simply filled in the information from Table B if it existed for that `blockid`. This is called a **LEFT JOIN**, since we're taking the table on the left (that is, Table A) and adding the information from Table B onto that. We could have also done a **RIGHT JOIN**, which does the same thing, except flipping the tables, giving us something that looks like:

| blockid | C000 | CA01|
|---|---|---|
|2|10|2|
|5|22|4|
|6|9|1|
|7|*null*|2|
|8|*null*|0|

# Applying Left Joins
When might you use left or right joins? Suppose you want to know which census blocks don't have any jobs. Then, we'd want to make sure that we keep all of the census blocks in the geography crosswalk, even if they aren't present in the workplace area characteristics tables, and add in the workplace area characteristics. This would mean that any blocks with an `na` value in the `C000` column would be blocks without any jobs. 

To do this `JOIN`, we can use the `left_join` statement.

In [ ]:
df_left_join <- left_join(ca_xwalk, ca_wac_2015, by = c("tabblk2010" = "w_geocode"), keep = TRUE)  %>% 
    select (tabblk2010, w_geocode, C000, CA01, CA02, CA03, ctyname, cbsaname)

head(df_left_join)

This is very similar to what we've done already with `inner_join`, except we replace `inner_join` with `left_join`'s. When doing `left_join` and `right_join`s, make sure to keep track of which one is first. Here, `ca_xwalk` comes first, so that's the table on the "left" side.

### Full Join

A full join keeps all unique ids, then puts `NULL` if it isn't part of that table. This is similar to a `left_join` or `right_join`, except instead of only keeping all IDs from one table, it keeps them from both tables. Consider our example with Table A and Table B. We want to join them such that we get a table that looks like:

| blockid | C000 | CA01 |
|---|---|---|
|1|5|*null*|
|2|10|2|
|3|2|*null*|
|4|6|*null*|
|5|22|4|
|6|9|1|
|7|*null*|2|
|8|*null*|0|

In a way, it's like combining the `left` and `right_join`s so that we have all information from both tables.

### Applying Full Joins

Suppose we want to know which home and work census block contain jobs. We can use a `full_join` for that.

We need to read in the `ca_rac_S000_JT00_2015.csv` data.

In [ ]:
ca_rac_2015 <- read_csv("ca_rac_S000_JT00_2015.csv")

Now, we can join the two data frames.

In [ ]:
df_full_join <- full_join(ca_rac_2015, ca_wac_2015, by = c("h_geocode" = "w_geocode"))  %>% 
    select (h_geocode, C000.x, C000.y, CA01.x, CA02.y, CA03.x, CFA02) 

head(df_full_join)


This will let us see which census blocks contain values for both workplace characteristics and residence characteristics. 
> Note: R renamed the variables that are the same in both data frames. For example, `C000.x` belongs to `ca_rac_2015` and `C000.y` belongs to `ca_wac_2015`.

## Checkpoint 2 Types of Joins

Consider the following situations. How would you answer the question posed? What type of join should you use for each one? Which tables do you need to join? Try doing the join.

- How many census blocks in the state contain a workplace and how many don't? Recall that the geography crosswalk table contains information about the census blocks in the state.
- Which county contains the most census blocks containing a workplace?
- Which metropolitan/micropolitan area had the most census blocks containing a residence of a worker in the state?

In [ ]:
# How many census blocks in the state contain a workplace and how many don't?
# How many do contain a workplace?


In [ ]:
# checkpoint_2a.hint()

In [ ]:
# checkpoint_2a.solution()

In [ ]:
# How many don't contain a workplace?


In [ ]:
# checkpoint_2b.hint()

In [ ]:
# checkpoint_2b.solution()

In [ ]:
# which county contains the most census blocks containing a workplace?


In [ ]:
# checkpoint_2c.hint()

In [ ]:
# checkpoint_2c.solution()

In [ ]:
# Which metropolitan/micropolitan area had the most census blocks containing a residence of a worker in the state?


In [ ]:
# checkpoint_2d.hint()

In [ ]:
# checkpoint_2d.solution()

## Basic Aggregrations

Let's count the total number of census blocks within each county that do no have jobs. We can do this using the `df_left_join` table.

In [ ]:
total_na_jobs_by_county <- df_left_join %>%
    group_by(ctyname) %>%
    summarize(
        total_missing_jobs = sum(is.na(C000))
    ) %>%
    ungroup() %>% 
    arrange(desc(total_missing_jobs))

head(total_na_jobs_by_county)

Lets figure out how many jobs are in each county. The first thing we want to do is `group_by` `ctyname`. Then we want to use the `summarize` function to sum the total number jobs in the `C000` column. Finally, we need to use the `ungroup()` function because if we don't, future calculations will likely produce errors.  

In [ ]:
total_jobs_by_county <- df_inner_join %>%
    group_by(ctyname) %>%
    summarize(
        total_jobs = sum(C000)
    ) %>%
    ungroup()  %>%
    arrange(desc(total_jobs))

head(total_jobs_by_county)

Now that we have total amount of jobs by county, let's calculate the proportion of jobs by county and then order them in descending order. This can be accomplished by taking the existing data frame `total_jobs_by_county` then using the `arrange()` function to order the variable `total_jobs`. Then, we can use the `mutate()` function to add a column called `prop` and setting that equal to `total_jobs` divided by the sum of `total_jobs`. I multiplied the results by 100 to get the percentage.

In [ ]:
total_jobs_prop <- total_jobs_by_county %>%
    arrange(desc(total_jobs)) %>%
    mutate(
        prop = total_jobs/sum(total_jobs)*100) 
head(total_jobs_prop)

## <font color = 'green'>Visualizations using `qplot`</font>
    
Visualizations can help us look at summaries of the data. They can also aid in providing good presentation aids.

In this section, we'll look at a few basic types of visualizations: histograms and scatterplots. In practice, there are many more types of visualizations. Furthermore, there are many different packages that can help you create these different visualizations. While we'll focus only on the basics here with the base R functions as well as `qplot` from the `ggplot2` package, keep in mind that there are many tools available.

### Making a Histogram
We'll start with a basic histogram. A histogram is useful for looking at the distribution of a numerical variable (that is, a variable that takes numbers as its values as opposed to categories). They can visually represent the spread and center of the variable, as well as provide insights to the shape of the distribution. For example, we can look at the number of **modes**, or peaks, in the data, which wouldn't be captured in numerical summaries such as mean or variance. We can also get a better idea of the **skew** of the data, determining whether most units are grouped on the left or right side (lower or higher values), or whether it's mostly symmetrical. 

We'll first look at the simplest way to make a graph, then show how to customize it using the plotting parameters as well as Figure and Axes objects. Let's look at a histogram of the total number of jobs by county in the 2015 Workplace Area Characteristics for California.

In [ ]:
qplot(total_jobs_prop$total_jobs, xlim = c(0,2000000), ylim = c(0,13))

The resulting histogram show the data are skewed right which means that the mean is greater than the median. This occurs because data skewed to the right have a few large values that increase the mean  but do not affect where the the median is. We can confirm that the mean is greater than the median by running the following code:

In [ ]:
mean(total_jobs_prop$total_jobs) > median(total_jobs_prop$total_jobs)

### Making a Scatterplot

When looking at two numerical variables to see if there is a correlation, we might want to use a scatterplot to visualize them. We can simply use the base R `qplot` function as before, passing it the x and y variables as the arguments. 

Let's look at an example looking at the relationship between number of jobs for two different age groups.

In [ ]:
qplot(df_inner_join$CA01, df_inner_join$CA02)

## Checkpoint 3: Visualize Your Datas

Using the methods we've described above, try visualizing data from a different state. What do the visualizations tell you? How are they different from the data from California? How are they the same? Does this make sense?

In [ ]:
# First, read in IL data.
il_data <- read_csv("il_wac_S000_JT00_2015.csv")


In [ ]:
# checkpoint_3.hint()

In [ ]:
# checkpoint_3.solution()

# Resources
Understanding Joins

<center> <img style="float: center;" src="join-venn.png" width=1000>


<center> <img style="float: center;" src="joins.png" width="700">
